In [30]:
import pandas as pd
import numpy as np
from os import listdir

In [31]:
baseline = 0.4
owner = 'MYMINT🐰'

In [32]:
df = pd.read_csv('OUTPUT.csv')
loveKeyword = {i.strip() for i in open('model/love.txt')}
hateKeyword = {i.strip() for i in open('model/hate.txt')}
def matchKeyword(keywords, message):
    n = 0
    for key in keywords:
        n += message.count(key)
    return n
def preprocess(df):
    df['datetime'] = pd.to_datetime(df['date']+' '+df['time'])
    df['love'] = df['message'].apply(lambda x: matchKeyword(loveKeyword, x))
    df['hate'] = df['message'].apply(lambda x: matchKeyword(hateKeyword, x))
preprocess(df)

In [33]:
def getRightSideWeight(df):
    return len(df[df['name'] == owner])/len(df)

def getAverageReplyTime(df):
    reply = df[['name', 'datetime']]
    currentReply = reply['name'][0]
    currentTime = reply['datetime'][0]
    times = 0
    time = 0
    for index, row in reply.iterrows():
        if currentReply != row['name']:
            timedelta = (row['datetime'] - currentTime).value/60e9
            # assume 24 hrs is end of talking topic
            if timedelta < 24*60:
                time += timedelta
                times += 1
        currentTime = row['datetime']
        currentReply = row['name']
    return time / times

def getLoveWordingRatio(df):
    love = np.sum(df[df['name'] != owner]['love'])
    hate = np.sum(df[df['name'] != owner]['hate'])
    return love/(love+hate)

In [38]:
reply_time_prop = 0.2
right_side_weight_prop = 0.2
wording_prop = 0.2
def predict(df):
    good = baseline
    
    right_side_weight = getRightSideWeight(df)
    right_side_predict = 2 * (0.5-right_side_weight) * right_side_weight_prop
    print(right_side_weight, right_side_predict)
    good += right_side_predict

    reply_time = getAverageReplyTime(df)
    reply_time_predict = (-1 * min((reply_time-5)/20, reply_time_prop)) if reply_time > 5 else min((5-reply_time)/20 ,reply_time_prop)
    print(reply_time, reply_time_predict)
    good += reply_time_predict
        
    love_word = getLoveWordingRatio(df)
    love_word_predict = 2 * (love_word-0.5) * wording_prop
    print(love_word, love_word_predict)
    good += love_word_predict
    
    return 0 if good < 0 else 1 if good > 1 else good

In [39]:
predict(df)

0.521415270018622 -0.008566108007448793
48.27272727272727 -0.2
0.375 -0.05


0.14143389199255124